In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm
from openmmtools.constants import kB

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



In [7]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/76/"
phase = 'complex'
sim_number = 2
old_aa_name = 'asn'
new_aa_name = 'lys'
length = 1
cache = 1
cache_length = 1

## Run neq integrator

In [3]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

# Define simulation parameters
# nsteps_eq = 25000 # 100 ps 
nsteps_eq = 1
nsteps_neq = 250000
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
cache_length = 1
temperature = 298.0 * unit.kelvin
kT = kB * temperature


In [8]:
# Read in vanilla htf
i = os.path.basename(os.path.dirname(outdir))
with open(os.path.join("/data/chodera/zhangi/perses_benchmark/neq/15/76/", f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)




In [9]:
# Read in lambda = 0 cache
with open(os.path.join(outdir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number]
system = htf.hybrid_system


In [11]:
# Read in lambda = 0 cache box vectors
with open(os.path.join(outdir, f"box_vectors.npy"), 'rb') as f:
    subset_box_vectors = np.load(f)
box_vectors = subset_box_vectors[sim_number][0]


In [13]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)


In [20]:
context.setPeriodicBoxVectors(*box_vectors[0])
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

# # Minimize
# openmm.LocalEnergyMinimizer.minimize(context)



In [21]:
context.getSystem().getSystem

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x2b15371a17b0> >

In [22]:
box_vectors[0]

array([[13.491772 ,  0.       ,  0.       ],
       [-4.4972568, 12.720165 ,  0.       ],
       [-4.4972568, -6.3600817, 11.015986 ]], dtype=float32)